# <font color='red'><ins>Práctica 3 FAA - Clasificación mediante Algoritmos Genéticos</ins></font>
## Alejandro Santorum Varela - alejandro.santorum@estudiante.uam.es
## José Manuel Chacón Aguilera - josem.chacon@estudiante.uam.es

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [26]:
# My modules
from Datos import Datos
import EstrategiaParticionado
import Clasificador

# External modules
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

# Important to pre-install tabulate (pip install tabulate)
from tabulate import tabulate

# Init class Classifier
clasificador = Clasificador.Clasificador()

In [27]:
db_titanic = Datos('../ConjuntosDatos/titanic.data', allNominal = True)
n,m = db_titanic.datos.shape
print("Numero de ejemplos:", n,"| Numero de columnas:", m)

datosenc = Datos.encodeDatos(db_titanic.datos)
n,m = datosenc.shape
print("Numero de ejemplos:", n,"| Numero de columnas (ENCODE):", m)

Numero de ejemplos: 1046 | Numero de columnas: 4
Numero de ejemplos: 1046 | Numero de columnas (ENCODE): 22


In [28]:
print(db_titanic.datos[0])
print(datosenc[0])

[1. 1. 3. 1.]
[0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [ ]:
# Entrenamiento. Consiste en sacar el individuo "ÓPTIMO".

    # - Definir la función/funciones de cruce
    # - Definir la mutación/mutaciones
    # - Definir la función/funciones de adaptación
    
    # ------------> Si nos calentamos, implementamos varias de estas y las consideramos hiper-parámetros del algoritmo.
    
# Clasificación. Consiste en comparar los genes de tu ejemplo con el individuo obtenido anteriormente, si tiene una codificación
# genética "parecida" codificas como superviviente.
    # Definir que es "parecido". 
